# CNN 복습
- 원핫인코딩 없이 sparse_categorical_crossentropy 사용

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0].reshape(28, 28), cmap='gray_r')
plt.show()

In [ ]:
X = tf.keras.Input(shape=[28, 28, 1])
H = tf.keras.layers.Conv2D(6, 5, padding="same", activation='swish')(X)
H = tf.keras.layers.MaxPool2D()(H)
H = tf.keras.layers.Conv2D(16, 5, activation='swish')(H)
H = tf.keras.layers.MaxPool2D()(H)
H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(120, activation="swish")(H)
H = tf.keras.layers.Dense(84, activation="swish")(H)
Y = tf.keras.layers.Dense(10, activation="softmax")(H)
model = tf.keras.Model(X, Y)
model.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")
# model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

- Maxpooling 대신 convolution stride=2를 이용하여 사이즈 줄이는 기법

In [ ]:
X = tf.keras.Input(shape=[28, 28, 1])
H = tf.keras.layers.Conv2D(32, 3, 2, activation='swish')(X)
H = tf.keras.layers.Conv2D(64, 3, 2, activation='swish')(H)
H = tf.keras.layers.Conv2D(128, 3, activation='swish')(H)
H = tf.keras.layers.Flatten()(H)
Y = tf.keras.layers.Dense(10, activation="softmax")(H)
model1 = tf.keras.Model(X, Y)
model1.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")
model1.summary()

In [ ]:
model1.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

# TF Dataset 사용법


In [ ]:
datasets = tf.data.Dataset.from_tensor_slices((x_train, y_train))
datasets = datasets.shuffle(1000).batch(128)

In [ ]:
x_batch, y_batch = next(iter(datasets))
print(x_batch.shape, y_batch.shape)
print(y_batch[:10])

In [ ]:
model.fit(datasets, epochs=10)

In [ ]:
for e in range(10):
    print(f"{e} epoch")
    for i, (x_batch, y_batch) in enumerate(datasets):
        result = model.train_on_batch(x_batch, y_batch)
        if i % 50 == 0:
            print(f"{i}th, {result}")
    model.evaluate(x_test, y_test)
    print()

# python multi assign & zip 

In [ ]:
def aa():
    return (1, 2), (3, 4)

(a, b), (c, d) = aa()
print(a, b, c, d)

In [ ]:
alist = [1, 2, 3, 4]

for i, a in enumerate(alist):
    print(i, a)

print(list(enumerate(alist)))

In [ ]:
bdict = {"a": 1, "b": 2, "c": 3}

for k, v in bdict.items():
    print(k, v)

print(list(bdict.items()))

In [ ]:
a = [1, 2, 3, 4]
b = ["a", "b", "c", "d"]
c = [6, 7, 8, 9]
for d, e, f in zip(a, b, c):
    print(d, e, f)

print(list(zip(a, b, c)))

# GradientTape 사용하기

In [ ]:
model = tf.keras.Model(X, Y)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

for e in range(10):
    print(f"{e} epoch")
    for i, (x_batch, y_batch) in enumerate(datasets):
        with tf.GradientTape() as tape:
            y_pred = model(x_batch, training=True)
            loss = loss_fn(y_batch, y_pred)
            grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        accuracy.update_state(y_batch, y_pred)

        if i % 50 == 0:
            print(f"{i} batch, {loss}, {accuracy.result()}")

In [ ]:
model = tf.keras.Model(X, Y)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred = model(x_batch, training=True)
        loss = loss_fn(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    accuracy.update_state(y_batch, y_pred)
    return loss, accuracy.result()

for e in range(10):
    print(f"{e} epoch")
    for i, (x_batch, y_batch) in enumerate(datasets):
        loss, acc = train_step(x_batch, y_batch)
        if i % 50 == 0:
            print(f"{i} batch, {loss}, {accuracy.result()}")

# Custom model class

In [ ]:
def make_cnn():
    X = tf.keras.Input(shape=[28, 28, 1])
    H = tf.keras.layers.Conv2D(32, 5, 2, activation='swish')(X)
    H = tf.keras.layers.Conv2D(64, 5, 2, activation='swish')(H)
    H = tf.keras.layers.Conv2D(128, 5, activation='swish')(H)
    H = tf.keras.layers.Flatten()(H)
    Y = tf.keras.layers.Dense(10, activation="softmax")(H)
    return tf.keras.Model(X, Y)

In [ ]:
class Custom(tf.keras.Model):
    def __init__(self, cnn):
        super(Custom, self).__init__()
        self.compile()
        self.cnn = cnn
        self.optimizer = tf.keras.optimizers.Adam()
        self.custom_loss = tf.keras.losses.SparseCategoricalCrossentropy()
        self.loss_metric = tf.keras.metrics.Mean()
        self.acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    
    def update_metrics(self, loss, y_batch, y_pred):
        self.loss_metric.update_state(loss)
        self.acc_metric.update_state(y_batch, y_pred)

    def train_step(self, batch):
        x_batch, y_batch = batch
        with tf.GradientTape() as tape:
            y_pred = self.cnn(x_batch, training=True)
            loss = self.custom_loss(y_batch, y_pred)
            grads = tape.gradient(loss, self.cnn.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.cnn.trainable_weights))

        self.update_metrics(loss, y_batch, y_pred)
        return {"loss": self.loss_metric.result(), "accuracy": self.acc_metric.result()}

In [ ]:
cnn = make_cnn()
model = Custom(cnn)
model.fit(datasets, epochs=10)

# CIFAR-10

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape, y_train.shape)

import matplotlib.pyplot as plt
plt.imshow(x_train[1])
plt.show()

In [ ]:
def make_cnn():
    X = tf.keras.Input(shape=[32, 32, 3])

    H = tf.keras.layers.Conv2D(32, 5, 2)(X)
    H = tf.keras.layers.BatchNormalization()(H)
    H = tf.keras.layers.Activation("swish")(H)

    H = tf.keras.layers.Conv2D(64, 5, 2)(H)
    H = tf.keras.layers.BatchNormalization()(H)
    H = tf.keras.layers.Activation("swish")(H)

    H = tf.keras.layers.Conv2D(128, 5)(H)
    H = tf.keras.layers.BatchNormalization()(H)
    H = tf.keras.layers.Activation("swish")(H)

    H = tf.keras.layers.Flatten()(H)
    Y = tf.keras.layers.Dense(10, activation="softmax")(H)
    return tf.keras.Model(X, Y)

In [ ]:
class Custom(tf.keras.Model):
    def __init__(self, cnn):
        super(Custom, self).__init__()
        self.compile()
        self.cnn = cnn
        self.optimizer = tf.keras.optimizers.Adam()
        self.custom_loss = tf.keras.losses.SparseCategoricalCrossentropy()
        self.loss_metric = tf.keras.metrics.Mean()
        self.acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    
    def update_metrics(self, loss, y_batch, y_pred):
        self.loss_metric.update_state(loss)
        self.acc_metric.update_state(y_batch, y_pred)

    def train_step(self, batch):
        x_batch, y_batch = batch
        with tf.GradientTape() as tape:
            y_pred = self.cnn(x_batch, training=True)
            loss = self.custom_loss(y_batch, y_pred)
            grads = tape.gradient(loss, self.cnn.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.cnn.trainable_weights))

        self.update_metrics(loss, y_batch, y_pred)
        return {"loss": self.loss_metric.result(), "accuracy": self.acc_metric.result()}
        
    def test_step(self, batch):
        x_batch, y_batch = batch
        y_pred = self.cnn(x_batch)
        loss = self.custom_loss(y_batch, y_pred)
        self.update_metrics(loss, y_batch, y_pred)
        return {"loss": self.loss_metric.result(), "accuracy": self.acc_metric.result()}

In [ ]:
cnn = make_cnn()
model = Custom(cnn)
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

In [ ]:
datasets = tf.data.Dataset.from_tensor_slices((x_train, y_train))
datasets = datasets.shuffle(1000).batch(128)

valid = tf.data.Dataset.from_tensor_slices((x_test, y_test))
valid = valid.shuffle(1000).batch(128)

In [ ]:
cnn = make_cnn()
model = Custom(cnn)
model.fit(datasets, epochs=10, validation_data=valid)

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
    dy = g.gradient(y, x)
print(dy)

# vanillaGAN

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train - 127.5) / 127.5
print(x_train.shape, x_train.min(), x_train.max())


In [ ]:
def make_discriminator():
    dx = tf.keras.Input(shape=[28 * 28])
    dh = tf.keras.layers.Dense(128)(dx)
    dh = tf.keras.layers.LeakyReLU(alpha=0.2)(dh)
    dh = tf.keras.layers.Dense(128)(dh)
    dh = tf.keras.layers.LeakyReLU(alpha=0.2)(dh)
    dy = tf.keras.layers.Dense(1, activation='sigmoid')(dh)
    return tf.keras.Model(dx, dy)

def make_generator():
    gx = tf.keras.Input(shape=[100])
    gh = tf.keras.layers.Dense(128)(gx)
    gh = tf.keras.layers.BatchNormalization()(gh)
    gh = tf.keras.layers.Activation('swish')(gh)
    gh = tf.keras.layers.Dense(128)(gx)
    gh = tf.keras.layers.BatchNormalization()(gh)
    gh = tf.keras.layers.Activation('swish')(gh)
    gy = tf.keras.layers.Dense(28 * 28, activation='tanh')(gh)
    return tf.keras.Model(gx, gy)


In [ ]:
class VanillaGAN(tf.keras.Model):
    def __init__(self, g, d):
        super(VanillaGAN, self).__init__()
        self.compile()

        self.generator = g
        self.discriminator = d

        self.d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
        self.g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

        self.d_loss_metrics = tf.keras.metrics.Mean()
        self.d_acc_metrics = tf.keras.metrics.BinaryAccuracy()
        self.g_loss_metrics = tf.keras.metrics.Mean()
        self.g_acc_metrics = tf.keras.metrics.BinaryAccuracy()

    def disc_loss(self, y_real, y_fake):
        real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(y_real), y_real)
        fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(y_fake), y_fake)
        return real_loss + fake_loss
    
    def gen_loss(self, y_fake):
        g_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(y_fake), y_fake)
        return g_loss

    def update_metrics(self, y_real, y_fake, g_loss, d_loss):
        self.d_loss_metrics.update_state(d_loss)
        self.d_acc_metrics.update_state(tf.ones_like(y_real), y_real)
        self.d_acc_metrics.update_state(tf.zeros_like(y_fake), y_fake)
        self.g_loss_metrics.update_state(g_loss)
        self.g_acc_metrics.update_state(tf.ones_like(y_fake), y_fake)

    def train_step(self, real):
        noise = tf.random.normal([tf.shape(real)[0], 100])
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            fake = self.generator(noise, training=True)
            y_real = self.discriminator(real, training=True)
            y_fake = self.discriminator(fake, training=True)

            g_loss = self.gen_loss(y_fake)
            d_loss = self.disc_loss(y_real, y_fake)

            g_grad = g_tape.gradient(g_loss, self.generator.trainable_variables)    
            d_grad = d_tape.gradient(d_loss, self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(g_grad, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(d_grad, self.discriminator.trainable_variables))

        self.update_metrics(y_real, y_fake, g_loss, d_loss)
        return {
            "d_loss": self.d_loss_metrics.result(),
            "d_acc": self.d_acc_metrics.result(),
            "g_loss": self.g_loss_metrics.result(),
            "g_acc": self.g_acc_metrics.result(),
        }

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class Monitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        x_rnd = np.random.randn(5 * 100).reshape(5, 100)
        generated = self.model.generator(x_rnd)
        for i in range(5):
            plt.subplot(1, 5, 1 + i)
            plt.axis('off')
            plt.imshow(generated[i].numpy().reshape(28, 28), cmap='gray_r')
        plt.show()

In [ ]:
discriminator = make_discriminator()
generator = make_generator()
gen = VanillaGAN(generator, discriminator)
gen.fit(x_train.reshape(-1, 28 * 28), epochs=50, batch_size=128, callbacks=[Monitor()])

# 작은 차원의 분포 학습

In [ ]:
class VanillaGAN(tf.keras.Model):
    def __init__(self, g, d):
        super(VanillaGAN, self).__init__()
        self.compile()

        self.generator = g
        self.discriminator = d

        self.d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
        self.g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

        self.d_loss_metrics = tf.keras.metrics.Mean()
        self.d_acc_metrics = tf.keras.metrics.BinaryAccuracy()
        self.g_loss_metrics = tf.keras.metrics.Mean()
        self.g_acc_metrics = tf.keras.metrics.BinaryAccuracy()

    def disc_loss(self, y_real, y_fake):
        real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(y_real), y_real)
        fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(y_fake), y_fake)
        return real_loss + fake_loss
    
    def gen_loss(self, y_fake):
        g_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(y_fake), y_fake)
        return g_loss

    def update_metrics(self, y_real, y_fake, g_loss, d_loss):
        self.d_loss_metrics.update_state(d_loss)
        self.d_acc_metrics.update_state(tf.ones_like(y_real), y_real)
        self.d_acc_metrics.update_state(tf.zeros_like(y_fake), y_fake)
        self.g_loss_metrics.update_state(g_loss)
        self.g_acc_metrics.update_state(tf.ones_like(y_fake), y_fake)

    def train_step(self, real):
        noise = tf.random.normal([tf.shape(real)[0], 5])
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            fake = self.generator(noise, training=True)
            y_real = self.discriminator(real, training=True)
            y_fake = self.discriminator(fake, training=True)

            g_loss = self.gen_loss(y_fake)
            d_loss = self.disc_loss(y_real, y_fake)

            g_grad = g_tape.gradient(g_loss, self.generator.trainable_variables)    
            d_grad = d_tape.gradient(d_loss, self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(g_grad, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(d_grad, self.discriminator.trainable_variables))

        self.update_metrics(y_real, y_fake, g_loss, d_loss)
        return {
            "d_loss": self.d_loss_metrics.result(),
            "d_acc": self.d_acc_metrics.result(),
            "g_loss": self.g_loss_metrics.result(),
            "g_acc": self.g_acc_metrics.result(),
        }

In [ ]:
def make_discriminator():
    dx = tf.keras.Input(shape=[2])
    dh = tf.keras.layers.Dense(25, activation='relu')(dx)
    dy = tf.keras.layers.Dense(1, activation='sigmoid')(dh)
    return tf.keras.Model(dx, dy)

def make_generator():
    gx = tf.keras.Input(shape=[5])
    gh = tf.keras.layers.Dense(15, activation="swish")(gx)
    gy = tf.keras.layers.Dense(2)(gh)
    return tf.keras.Model(gx, gy)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class Monitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        x_rnd = np.random.randn(1000).reshape(200, 5)
        generated = self.model.generator(x_rnd)
        plt.scatter(x_train[:1000, 0], x_train[:1000, 1], color='red')
        plt.scatter(generated[:, 0], generated[:, 1], color="blue")
        plt.show()

In [ ]:
x_train = np.random.randn(120000).reshape(60000, 2)
print(x_train.shape)

x_train[:, 1] = 1 * x_train[:, 0] + 2 * x_train[:, 1]

plt.scatter(x_train[:1000, 0], x_train[:1000, 1], color='red')
plt.show()

In [ ]:
tf.keras.backend.clear_session()

generator = make_generator()
discriminator = make_discriminator()
gen = VanillaGAN(generator, discriminator)
gen.fit(x_train, epochs=20, batch_size=128, callbacks=[Monitor()])